In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
import argparse
import copy
# import data_loader
import pdb
import torch.nn.functional as F
import re, random, collections
import pickle

rd=3473 
torch.manual_seed(rd)
torch.cuda.manual_seed_all(rd)

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='1'

In [2]:
class args():
    
    # download tinyimagenet dataset and set the path
    data_path = "/data/dataset/tiny_imagenet/tiny-imagenet-200/"
    num_class = 200
    class_per_task = 20
    n_classes=class_per_task
    num_task = 10
    dataset = "tinyimagenet"
    train_batch = 128
    test_batch = 256
    workers = 16
    gamma = 0.2
    random_classes = False
    validation = 0
    memory = 2000
    mu = 1
    beta = 1.0
    r = 2
    overflow=False
    print_freq=100
    
    lr=0.01
    resume=False
    total_epoch=160
    model_path='ckpt816/vgg16-3conv816'

args=args()

In [3]:
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import argparse
# from utils import progress_bar
from torch.optim.lr_scheduler import MultiStepLR
torch.set_printoptions(precision=5,sci_mode=False)

In [4]:
class HetConv(nn.Module):
    def __init__(self, in_channels, out_channels, p,kernel,stride,padding):
        super(HetConv, self).__init__()
        # Groupwise Convolution
        self.gwc = nn.Conv2d(in_channels, out_channels, kernel_size=kernel, 
                             stride=stride, padding=padding, groups=int(p/gp), bias=True)
        # Pointwise Convolution
        self.pwc = nn.Conv2d(in_channels, out_channels, kernel_size=1, groups=int(p/pt),bias=True)

    def forward(self, x):
        return self.gwc(x) + self.pwc(x)
    
'''VGG11/13/16/19 in Pytorch.'''
import torch
import torch.nn as nn


cfg = {
    'VGG11': [64,'G', 'M', 128,'G', 'M', 256,'G', 256,'G', 'M', 512,'G', 512,'G', 'M', 512, 'G',512,'G', 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.features = self._make_layers(cfg['VGG16'])
        self.classifier = nn.Linear(2048, args.class_per_task)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1)]
                in_channels = x
                layers += [HetConv(in_channels,in_channels,in_channels,3,1,1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
        layers += [nn.AvgPool2d(kernel_size=2, stride=2)]
        return nn.Sequential(*layers)

In [5]:
def save_model(task,acc,model):
    print('Saving..')
    statem = {
        'net': model.state_dict(),
        'acc': acc,
    }
    fname=args.model_path
    if not os.path.isdir(fname):
        os.makedirs(fname)   
    torch.save(statem, fname+'/ckpt_task'+str(task)+'.pth')

        
def load_model(task,model):
    fname=args.model_path
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    print(fname+'/ckpt_task'+str(task)+'.pth')
    checkpoint = torch.load(fname+'/ckpt_task'+str(task)+'.pth')
    model.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    return best_acc

In [6]:
def train(train_loader,epoch,task,model):
    print('\nEpoch: %d' % epoch)
    model.train()
    global best_acc
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.cuda(), targets.cuda()
        targets=targets-task*args.n_classes
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, targets)#+ 1*regularizer_loss(model)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        acc = 100.*correct/total
    
    print("[Train: ], [%d/%d: ], [Accuracy: %f], [Loss: %f] [Lr: %f]" 
          %(epoch,args.total_epoch,acc, train_loss/batch_idx,optimizer.param_groups[0]['lr']))

In [7]:
def test(test_loader,task,model):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.cuda(), targets.cuda()
            targets=targets-task*args.n_classes
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    acc = 100.*correct/total
    print("[Test Accuracy: %f], [Loss: %f]" %(acc,test_loss/batch_idx))
    
    if acc>best_acc:
        save_model(task,acc,model)
        best_acc=acc
    return acc

In [8]:
def grad_false(modelm):
    gradf=[0,1,8,9,16,17,24,25,32,33,40,41,48,49,56,57,64,65,72,73]

    i=0
    for p in modelm.parameters():
        if i in gradf:
            p.requires_grad=False
        i=i+1


In [9]:
import incremental_dataloader as data
inc_dataset = data.IncrementalDataset(
                                dataset_name=args.dataset,
                                args = args,
                                random_order=args.random_classes,
                                shuffle=True,
                                seed=1,
                                batch_size=args.train_batch,
                                workers=args.workers,
                                validation_split=args.validation,
                                increment=args.class_per_task,
                            )
task_data=[]
for i in range(args.num_task):
    task_info, train_loader, val_loader, test_loader = inc_dataset.new_task()
    task_data.append([train_loader,test_loader])

0
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
1
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
2
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
3
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
4
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
5
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
6
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
7
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
8
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
9
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20]


In [10]:
gp=8
pt=16


import incremental_dataloader as data
task_acc=[]
for task in range(0,args.num_task):
    print('Training Task :---'+str(task)+'\n')
    best_acc=0
    
    if task==0:
        
        train_loader= task_data[task][0]
        test_loader = task_data[task][1]
        modelm = Net().cuda()
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(modelm.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)
        schedulerG = MultiStepLR(optimizer, milestones=[70, 100,120], gamma=0.1)

        for epoch in range(args.total_epoch):
            train(train_loader,epoch,task,modelm)
            test(test_loader,task,modelm)
            schedulerG.step()
            
        acc1=load_model(task,modelm)
    
    
    
    if task>=1:
        train_loader= task_data[task][0]
        test_loader = task_data[task][1]
        
        modelm = Net().cuda()
        
        acc1=load_model(task-1,modelm)  
        grad_false(modelm)
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(modelm.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)
        schedulerG = MultiStepLR(optimizer, milestones=[70, 100,120], gamma=0.1)

        for epoch in range(args.total_epoch):
            train(train_loader,epoch,task,modelm)
            test(test_loader,task,modelm)
            schedulerG.step()
            
        acc1=load_model(task,modelm)
        
    task_acc.append(acc1)
    np.save(args.model_path+'.npy',task_acc)
    print('Task: '+str(task)+'  Test_accuracy1: '+ str(acc1))

print(task_acc)

Training Task :---0


Epoch: 0
[Train: ], [0/160: ], [Accuracy: 16.980000], [Loss: 2.811471] [Lr: 0.010000]
[Test Accuracy: 18.500000], [Loss: 4.500120]
Saving..

Epoch: 1
[Train: ], [1/160: ], [Accuracy: 22.260000], [Loss: 2.684302] [Lr: 0.010000]
[Test Accuracy: 25.800000], [Loss: 3.199207]
Saving..

Epoch: 2
[Train: ], [2/160: ], [Accuracy: 26.750000], [Loss: 2.520357] [Lr: 0.010000]
[Test Accuracy: 28.200000], [Loss: 3.073271]
Saving..

Epoch: 3
[Train: ], [3/160: ], [Accuracy: 26.980000], [Loss: 2.505863] [Lr: 0.010000]
[Test Accuracy: 28.200000], [Loss: 3.110817]

Epoch: 4
[Train: ], [4/160: ], [Accuracy: 29.980000], [Loss: 2.402824] [Lr: 0.010000]
[Test Accuracy: 33.600000], [Loss: 2.902164]
Saving..

Epoch: 5
[Train: ], [5/160: ], [Accuracy: 31.940000], [Loss: 2.306216] [Lr: 0.010000]
[Test Accuracy: 34.500000], [Loss: 3.006265]
Saving..

Epoch: 6
[Train: ], [6/160: ], [Accuracy: 32.840000], [Loss: 2.271975] [Lr: 0.010000]
[Test Accuracy: 33.800000], [Loss: 2.868304]

Epoch: 7


[Train: ], [60/160: ], [Accuracy: 66.390000], [Loss: 1.085273] [Lr: 0.010000]
[Test Accuracy: 62.400000], [Loss: 1.910198]
Saving..

Epoch: 61
[Train: ], [61/160: ], [Accuracy: 68.310000], [Loss: 1.026966] [Lr: 0.010000]
[Test Accuracy: 60.800000], [Loss: 2.089266]

Epoch: 62
[Train: ], [62/160: ], [Accuracy: 68.240000], [Loss: 1.050946] [Lr: 0.010000]
[Test Accuracy: 57.100000], [Loss: 2.224044]

Epoch: 63
[Train: ], [63/160: ], [Accuracy: 67.740000], [Loss: 1.059745] [Lr: 0.010000]
[Test Accuracy: 61.400000], [Loss: 2.017621]

Epoch: 64
[Train: ], [64/160: ], [Accuracy: 66.340000], [Loss: 1.078418] [Lr: 0.010000]
[Test Accuracy: 57.300000], [Loss: 2.142332]

Epoch: 65
[Train: ], [65/160: ], [Accuracy: 67.900000], [Loss: 1.051748] [Lr: 0.010000]
[Test Accuracy: 58.400000], [Loss: 2.249557]

Epoch: 66
[Train: ], [66/160: ], [Accuracy: 68.320000], [Loss: 1.051784] [Lr: 0.010000]
[Test Accuracy: 61.000000], [Loss: 1.961890]

Epoch: 67
[Train: ], [67/160: ], [Accuracy: 68.390000], [Loss: 

[Test Accuracy: 67.100000], [Loss: 1.822861]

Epoch: 121
[Train: ], [121/160: ], [Accuracy: 83.520000], [Loss: 0.551234] [Lr: 0.000010]
[Test Accuracy: 66.600000], [Loss: 1.839215]

Epoch: 122
[Train: ], [122/160: ], [Accuracy: 83.150000], [Loss: 0.550439] [Lr: 0.000010]
[Test Accuracy: 66.800000], [Loss: 1.841831]

Epoch: 123
[Train: ], [123/160: ], [Accuracy: 83.180000], [Loss: 0.549508] [Lr: 0.000010]
[Test Accuracy: 66.700000], [Loss: 1.846780]

Epoch: 124
[Train: ], [124/160: ], [Accuracy: 83.530000], [Loss: 0.549672] [Lr: 0.000010]
[Test Accuracy: 67.100000], [Loss: 1.847011]

Epoch: 125
[Train: ], [125/160: ], [Accuracy: 83.060000], [Loss: 0.559974] [Lr: 0.000010]
[Test Accuracy: 66.900000], [Loss: 1.825260]

Epoch: 126
[Train: ], [126/160: ], [Accuracy: 83.040000], [Loss: 0.556081] [Lr: 0.000010]
[Test Accuracy: 66.800000], [Loss: 1.849753]

Epoch: 127
[Train: ], [127/160: ], [Accuracy: 82.810000], [Loss: 0.569594] [Lr: 0.000010]
[Test Accuracy: 67.300000], [Loss: 1.834105]

Ep

[Test Accuracy: 53.200000], [Loss: 1.924067]

Epoch: 20
[Train: ], [20/160: ], [Accuracy: 54.700000], [Loss: 1.425999] [Lr: 0.010000]
[Test Accuracy: 50.400000], [Loss: 2.024828]

Epoch: 21
[Train: ], [21/160: ], [Accuracy: 55.530000], [Loss: 1.433401] [Lr: 0.010000]
[Test Accuracy: 54.700000], [Loss: 1.924151]

Epoch: 22
[Train: ], [22/160: ], [Accuracy: 55.680000], [Loss: 1.408554] [Lr: 0.010000]
[Test Accuracy: 55.300000], [Loss: 1.885956]

Epoch: 23
[Train: ], [23/160: ], [Accuracy: 55.470000], [Loss: 1.417604] [Lr: 0.010000]
[Test Accuracy: 55.300000], [Loss: 1.879177]

Epoch: 24
[Train: ], [24/160: ], [Accuracy: 55.760000], [Loss: 1.398015] [Lr: 0.010000]
[Test Accuracy: 52.300000], [Loss: 1.936551]

Epoch: 25
[Train: ], [25/160: ], [Accuracy: 56.690000], [Loss: 1.383670] [Lr: 0.010000]
[Test Accuracy: 52.500000], [Loss: 2.003051]

Epoch: 26
[Train: ], [26/160: ], [Accuracy: 57.080000], [Loss: 1.377997] [Lr: 0.010000]
[Test Accuracy: 53.000000], [Loss: 2.044286]

Epoch: 27
[Train

[Train: ], [80/160: ], [Accuracy: 69.090000], [Loss: 0.985006] [Lr: 0.001000]
[Test Accuracy: 63.100000], [Loss: 1.592675]

Epoch: 81
[Train: ], [81/160: ], [Accuracy: 69.360000], [Loss: 0.973954] [Lr: 0.001000]
[Test Accuracy: 62.800000], [Loss: 1.610929]

Epoch: 82
[Train: ], [82/160: ], [Accuracy: 69.800000], [Loss: 0.969384] [Lr: 0.001000]
[Test Accuracy: 63.100000], [Loss: 1.616815]

Epoch: 83
[Train: ], [83/160: ], [Accuracy: 69.640000], [Loss: 0.970294] [Lr: 0.001000]
[Test Accuracy: 63.200000], [Loss: 1.639185]

Epoch: 84
[Train: ], [84/160: ], [Accuracy: 69.290000], [Loss: 0.979632] [Lr: 0.001000]
[Test Accuracy: 63.200000], [Loss: 1.620894]

Epoch: 85
[Train: ], [85/160: ], [Accuracy: 69.640000], [Loss: 0.981042] [Lr: 0.001000]
[Test Accuracy: 62.900000], [Loss: 1.624585]

Epoch: 86
[Train: ], [86/160: ], [Accuracy: 69.970000], [Loss: 0.964567] [Lr: 0.001000]
[Test Accuracy: 62.900000], [Loss: 1.610700]

Epoch: 87
[Train: ], [87/160: ], [Accuracy: 70.680000], [Loss: 0.951378]

...

[Test Accuracy: 63.300000], [Loss: 1.684680]

Epoch: 43
[Train: ], [43/160: ], [Accuracy: 64.160000], [Loss: 1.176960] [Lr: 0.010000]
[Test Accuracy: 63.300000], [Loss: 1.743565]

Epoch: 44
[Train: ], [44/160: ], [Accuracy: 64.300000], [Loss: 1.154690] [Lr: 0.010000]
[Test Accuracy: 63.200000], [Loss: 1.778312]

Epoch: 45
[Train: ], [45/160: ], [Accuracy: 64.110000], [Loss: 1.157191] [Lr: 0.010000]
[Test Accuracy: 61.700000], [Loss: 1.833841]

Epoch: 46
[Train: ], [46/160: ], [Accuracy: 64.390000], [Loss: 1.153056] [Lr: 0.010000]
[Test Accuracy: 64.800000], [Loss: 1.608368]

Epoch: 47
[Train: ], [47/160: ], [Accuracy: 64.910000], [Loss: 1.129368] [Lr: 0.010000]
[Test Accuracy: 60.800000], [Loss: 1.822640]

Epoch: 48
[Train: ], [48/160: ], [Accuracy: 64.940000], [Loss: 1.129973] [Lr: 0.010000]
[Test Accuracy: 63.300000], [Loss: 1.615266]

Epoch: 49
[Train: ], [49/160: ], [Accuracy: 65.010000], [Loss: 1.130864] [Lr: 0.010000]
[Test Accuracy: 65.400000], [Loss: 1.594398]

Epoch: 50
[Train

[Test Accuracy: 70.300000], [Loss: 1.372739]

Epoch: 104
[Train: ], [104/160: ], [Accuracy: 74.290000], [Loss: 0.851685] [Lr: 0.000100]
[Test Accuracy: 70.800000], [Loss: 1.364164]

Epoch: 105
[Train: ], [105/160: ], [Accuracy: 73.960000], [Loss: 0.837668] [Lr: 0.000100]
[Test Accuracy: 70.900000], [Loss: 1.366496]

Epoch: 106
[Train: ], [106/160: ], [Accuracy: 74.150000], [Loss: 0.846633] [Lr: 0.000100]
[Test Accuracy: 71.100000], [Loss: 1.371272]

Epoch: 107
[Train: ], [107/160: ], [Accuracy: 74.400000], [Loss: 0.833048] [Lr: 0.000100]
[Test Accuracy: 70.600000], [Loss: 1.366648]

Epoch: 108
[Train: ], [108/160: ], [Accuracy: 73.340000], [Loss: 0.862152] [Lr: 0.000100]
[Test Accuracy: 71.000000], [Loss: 1.372279]

Epoch: 109
[Train: ], [109/160: ], [Accuracy: 74.300000], [Loss: 0.842200] [Lr: 0.000100]
[Test Accuracy: 70.600000], [Loss: 1.379034]

Epoch: 110
[Train: ], [110/160: ], [Accuracy: 73.830000], [Loss: 0.852182] [Lr: 0.000100]
[Test Accuracy: 70.900000], [Loss: 1.361616]

Ep

[Test Accuracy: 59.200000], [Loss: 1.789187]
Saving..

Epoch: 3
[Train: ], [3/160: ], [Accuracy: 53.570000], [Loss: 1.442905] [Lr: 0.010000]
[Test Accuracy: 54.300000], [Loss: 1.868605]

Epoch: 4
[Train: ], [4/160: ], [Accuracy: 55.310000], [Loss: 1.393824] [Lr: 0.010000]
[Test Accuracy: 56.600000], [Loss: 1.740948]

Epoch: 5
[Train: ], [5/160: ], [Accuracy: 56.530000], [Loss: 1.351951] [Lr: 0.010000]
[Test Accuracy: 60.800000], [Loss: 1.665516]
Saving..

Epoch: 6
[Train: ], [6/160: ], [Accuracy: 56.880000], [Loss: 1.347573] [Lr: 0.010000]
[Test Accuracy: 57.900000], [Loss: 1.812636]

Epoch: 7
[Train: ], [7/160: ], [Accuracy: 58.190000], [Loss: 1.300920] [Lr: 0.010000]
[Test Accuracy: 60.200000], [Loss: 1.653518]

Epoch: 8
[Train: ], [8/160: ], [Accuracy: 58.660000], [Loss: 1.293651] [Lr: 0.010000]
[Test Accuracy: 57.200000], [Loss: 1.789326]

Epoch: 9
[Train: ], [9/160: ], [Accuracy: 58.990000], [Loss: 1.281526] [Lr: 0.010000]
[Test Accuracy: 62.300000], [Loss: 1.586481]
Saving..

Epo

[Test Accuracy: 66.500000], [Loss: 1.463547]

Epoch: 64
[Train: ], [64/160: ], [Accuracy: 69.630000], [Loss: 0.949016] [Lr: 0.010000]
[Test Accuracy: 60.600000], [Loss: 1.756972]

Epoch: 65
[Train: ], [65/160: ], [Accuracy: 68.840000], [Loss: 0.951944] [Lr: 0.010000]
[Test Accuracy: 66.200000], [Loss: 1.569104]

Epoch: 66
[Train: ], [66/160: ], [Accuracy: 68.910000], [Loss: 0.945541] [Lr: 0.010000]
[Test Accuracy: 62.300000], [Loss: 1.683094]

Epoch: 67
[Train: ], [67/160: ], [Accuracy: 69.670000], [Loss: 0.929598] [Lr: 0.010000]
[Test Accuracy: 60.100000], [Loss: 1.895646]

Epoch: 68
[Train: ], [68/160: ], [Accuracy: 70.100000], [Loss: 0.919165] [Lr: 0.010000]
[Test Accuracy: 62.000000], [Loss: 1.774370]

Epoch: 69
[Train: ], [69/160: ], [Accuracy: 68.990000], [Loss: 0.944663] [Lr: 0.010000]
[Test Accuracy: 62.100000], [Loss: 1.658521]

Epoch: 70
[Train: ], [70/160: ], [Accuracy: 72.510000], [Loss: 0.846403] [Lr: 0.001000]
[Test Accuracy: 68.000000], [Loss: 1.347084]
Saving..

Epoch: 

[Train: ], [124/160: ], [Accuracy: 78.220000], [Loss: 0.687909] [Lr: 0.000010]
[Test Accuracy: 69.800000], [Loss: 1.319965]

Epoch: 125
[Train: ], [125/160: ], [Accuracy: 77.560000], [Loss: 0.698503] [Lr: 0.000010]
[Test Accuracy: 69.300000], [Loss: 1.328349]

Epoch: 126
[Train: ], [126/160: ], [Accuracy: 77.470000], [Loss: 0.714993] [Lr: 0.000010]
[Test Accuracy: 69.300000], [Loss: 1.329501]

Epoch: 127
[Train: ], [127/160: ], [Accuracy: 77.550000], [Loss: 0.702792] [Lr: 0.000010]
[Test Accuracy: 69.900000], [Loss: 1.322331]

Epoch: 128
[Train: ], [128/160: ], [Accuracy: 77.620000], [Loss: 0.698826] [Lr: 0.000010]
[Test Accuracy: 70.000000], [Loss: 1.319780]

Epoch: 129
[Train: ], [129/160: ], [Accuracy: 77.780000], [Loss: 0.694740] [Lr: 0.000010]
[Test Accuracy: 69.300000], [Loss: 1.319229]

Epoch: 130
[Train: ], [130/160: ], [Accuracy: 77.640000], [Loss: 0.710721] [Lr: 0.000010]
[Test Accuracy: 70.100000], [Loss: 1.326753]

Epoch: 131
[Train: ], [131/160: ], [Accuracy: 77.300000], [

In [11]:
np.mean(task_acc)

67.78999999999999